# 使用Notebook

# 问题描述：具有任务依赖和不确定性的多智能体重叠联盟形成
本算法主要解决的是在一个存在资源约束、任务具有先后优先级依赖、以及对任务需求存在不确定性的环境中，如何分配一组异构智能体（Agents）去执行一组任务（Tasks）的问题。

### 核心特征

1.  **重叠联盟形成 (Overlapping Coalition Formation, OCF)**
    *   智能体可以同时加入多个联盟（即同时分配给多个任务部分资源），通过分配不同比例的资源参与不同的任务。
    *   每个任务由一个“联盟”负责执行。

2.  **不确定性环境**
    *   **资源需求不确定**：任务所需的资源量不是固定的，而是服从某种概率分布。智能体基于“信念（Belief）”来估计需求（例如使用分位数计算 `calculate_demand_quantile`）。
    *   **任务价值不确定**：任务的价值可能也是概率性的。

3.  **时空约束与同步机制（固定速度+等待模型）**
    *   **固定速度移动**：智能体以恒定速度 $v_i$ 飞行。
    *   **同步执行 (Synchronization)**：对于一个联盟中的所有成员，必须全部到达任务地点后才能开始执行任务。
    *   **先到等待**：早到达的智能体会在任务点悬停等待（Hovering），直到最晚的成员到达。
    *   **任务优先级**：任务具有优先级（Priority），智能体必须按优先级顺序依次执行分配给它的任务。

4.  **资源约束**
    *   每个智能体拥有固定总量的多种类型资源。
    *   分配给所有任务的资源总和不能超过其拥有的总量。

# 联盟效用与成本计算模型

智能体 $n$ 在任务 $m$ 的联盟 $C_m$ 中的个体效用 $u_n(C_m)$ 定义为期望收益减去预期成本。

## 1. 效用公式

$$
u_n(C_m) = \underbrace{r_n(C_m) \cdot V(C_m) \cdot D(C_m)}_{\text{Expected Revenue}} - \underbrace{Cost_n(C_m)}_{\text{Energy Cost}}
$$

其中：

*   **$r_n(C_m)$ (Contribution Ratio)**: 智能体在联盟中的贡献比例。
    $$
    r_n(C_m) = \frac{\| \mathbf{x}_{n \to m} \|}{\max(\sum_{i \in C_m} \| \mathbf{x}_{i \to m} \|, \epsilon)}
    $$
    *   $\mathbf{x}_{n \to m}$: 智能体 $n$ 分配给任务 $m$ 的资源向量。

*   **$V(C_m)$ (Expected Task Value)**: 任务的期望价值。
    $$
    V(C_m) = \sum_{v \in Values} (v \cdot Belief_n(v))
    $$
    *   基于智能体对任务价值的信念分布计算。

*   **$D(C_m)$ (Completion Degree)**: 任务的完成度（基于信念的期望需求）。
    $$
    D(C_m) = \frac{1}{Z_c} \sum_{k=1}^{K} \min \left( \frac{\sum_{i \in C_m} x_{i \to m}^{(k)}}{\hat{q}_m^{(k)}}, 1 \right)
    $$
    *   $\hat{q}_m^{(k)}$: 智能体估计的任务 $m$ 对资源类型 $k$ 的需求量（分位数估计）。
    *   $Z_c$: 有效资源类型的数量。

## 2. 成本模型 (Cost Model)

成本主要由能量消耗构成，采用了**固定速度+等待 (Fixed Speed + Waiting)** 模型。

$$
Cost_n(C_m) = t_{\text{fly}} \cdot \alpha_{\text{fly}} + t_{\text{wait}} \cdot \alpha_{\text{wait}} + t_{\text{exec}} \cdot \beta
$$

各个分项的计算如下：

1.  **飞行时间 ($t_{\text{fly}}$)**:
    $$
    t_{\text{fly}} = \frac{\text{Distance}}{v_n}
    $$
    *   智能体以固定速度 $v_n$ 从上一个位置移动到当前任务位置。

2.  **等待时间 ($t_{\text{wait}}$)**:
    $$
    t_{\text{wait}} = \max(0, T_{\text{start}}^{(m)} - T_{\text{arrival}}^{(n)})
    $$
    *   $T_{\text{start}}^{(m)}$: 任务 $m$ 的同步开始时间（即联盟中最后一个成员到达的时间）。
    *   $T_{\text{arrival}}^{(n)}$: 智能体 $n$ 到达任务地点的时间。
    *   **能耗系数**: $\alpha_{\text{wait}} = 0.5 \cdot \alpha_{\text{fly}}$ (悬停能耗约为飞行能耗的一半)。

3.  **执行时间 ($t_{\text{exec}}$)**:
    $$
    t_{\text{exec}} = \sum_{k \in \text{Assigned Resources}} \text{Duration}_k
    $$
    *   智能体实际参与执行的时间，消耗每单位时间的能耗 $\beta$。

## 3. 边际效用增益 (Marginal Utility Gain)

在结构自适应 (Structure Adaptation, SA) 过程中，评估从结构 $CS_{P}$ 变为 $CS_{Q}$ 的增益 $\Delta U$：

$$
\Delta U = \sum_{n \in N} \left( u_n(CS_{Q}) - u_n(CS_{P}) \right)
$$

这不仅包含智能体自身的效用变化，还可能包含对同一联盟中其他成员的外部性影响（如新的成员加入改变了贡献比、完成度或同步时间）。

在算法中，为了判定一个智能体 $n$ 是否应该从当前的联盟结构 $S_{P}$ 迁移到新的联盟结构 $S_{Q}$（例如：加入新任务、离开旧任务或调整资源分配），我们比较两个状态下的**帕累托改进（Pareto Improvement）**或某种形式的**社会福利（Social Welfare）**增量。

$$
\Delta U (S_{Q}, S_{P}) = LHS(S_{Q}, S_{P}) - RHS(S_{Q}, S_{P})
$$

如果 $\Delta U > 0$，则认为结构 $S_{Q}$ 优于 $S_{P}$。公式可以分解为以下三部分：

### 1. 自身的效用变化 (Self Utility Change)

这是最直接的动力，即智能体 $n$ 在新结构下获得的总效用是否高于旧结构。

$$
\Delta U_{self} = \sum_{m \in A_n(S_Q)} u_n^{(m)}(S_Q) - \sum_{m \in A_n(S_P)} u_n^{(m)}(S_P)
$$
*   $A_n(S)$: 智能体 $n$ 在结构 $S$ 中参与的任务集合。

### 2. 目标任务与源任务成员的效用影响 (Effect on Goal/Source Task Members)

当智能体 $n$ 加入一个新的任务 $A_{new}$（目标任务）或离开一个旧任务 $A_{old}$（源任务）时，会导致该任务联盟中**其他成员**的效用发生变化（例如：贡献比例稀释、完成度提高、同步等待时间改变）。

*   **对于新加入的任务** $A_{new} \in (A_n(S_Q) \setminus A_n(S_P))$：
    $$
    \Delta U_{new} = \sum_{k \in Members(A_{new}) \setminus \{n\}} \left( u_k^{(A_{new})}(S_Q) - u_k^{(A_{new})}(S_P) \right)
    $$
*   **对于离开的任务** $A_{old} \in (A_n(S_P) \setminus A_n(S_Q))$：
    $$
    \Delta U_{old} = \sum_{k \in Members(A_{old}) \setminus \{n\}} \left( u_k^{(A_{old})}(S_P) - u_k^{(A_{old})}(S_Q) \right)
    $$

### 3. 相关任务成员的外部性 (Externality on Related Members)

除了直接变动的任务外，智能体 $n$ 的行为还可能间接影响到与其在**其他共同任务**中合作的伙伴。为了全面评估，算法计算了与 $n$ 有关联的所有成员（$Mem(A(n)$)）在所有任务上的总效用变化。

$$
\Delta U_{related} = \sum_{o \in Mem(A_n) \setminus \{n\}} \left( \sum_{m \in Tasks(o)} u_o^{(m)}(S_Q) - \sum_{m \in Tasks(o)} u_o^{(m)}(S_P) \right)
$$

### 综合判定公式

最终的判定逻辑是将上述部分整合为 LHS（左式，代表新结构的优势）和 RHS（右式，代表旧结构的优势）：

$$
\begin{aligned}
LHS &= \underbrace{u_n(S_Q)}_{\text{Self Q}} + \underbrace{\sum_{g \in Mem(New)} \Delta u_g(New)}_{\text{Impact on New Task}} + \underbrace{\sum_{o \in Mem(A_n)} u_o(S_Q)}_{\text{Related Members Q}} \\
RHS &= \underbrace{u_n(S_P)}_{\text{Self P}} + \underbrace{\sum_{h \in Mem(Old)} \Delta u_h(Old)}_{\text{Impact on Old Task}} + \underbrace{\sum_{o \in Mem(A_n)} u_o(S_P)}_{\text{Related Members P}}
\end{aligned}
$$

通过计算 $\Delta U = LHS - RHS$，算法能够捕捉到局部操作对全局或局部社区福利的综合影响，从而引导系统向更优的联盟结构演化。